In [1]:
import pydwf
import pydwf.utilities
import numpy as np
import matplotlib.pyplot as plt
import time
import serial
import datetime
import json
from pathlib import Path

%matplotlib widget

# Open AD2 Device

In [2]:
def scorefunc(x):
    return x[pydwf.DwfEnumConfigInfo.AnalogInBufferSize]

dwf = pydwf.DwfLibrary()
dev = pydwf.utilities.openDwfDevice(dwf,
    score_func=scorefunc
)

# Setup AD2

## Setup Input

In [134]:
inCH = [0, 1]
inFreq = 1e6
inRecordLen = 10e-3
inNSamples = int(inFreq * inRecordLen)

anIn = dev.analogIn
anIn.reset()
for CH in inCH:
    anIn.channelEnableSet(CH, True)
    
anIn.channelRangeSet(0, 3.5)
anIn.channelOffsetSet(0, 1.65)
    
anIn.channelRangeSet(0, 30)
anIn.channelOffsetSet(0, 0)

anIn.acquisitionModeSet(pydwf.DwfAcquisitionMode.Record)
anIn.frequencySet(inFreq)
anIn.recordLengthSet(inRecordLen)
anIn.triggerSourceSet(pydwf.DwfTriggerSource.PC)
time.sleep(1)

## Setup Supply

In [4]:
supplyV = 3.3

anIO = dev.analogIO
anIO.reset()
anIO.enableSet(True)
anIO.channelNodeSet(0, 0, True) # Enable positive supply
anIO.channelNodeSet(0, 1, supplyV) # Set positive supply voltage
anIO.configure()

# Read data

In [ ]:
with serial.Serial("/dev/ttyUSB0", 115200, timeout=1) as ser:
    while True:
        ser.write(b'p')
        time.sleep(1)

In [143]:
now = datetime.datetime.now()
dist = 107.46
outpath = Path("data", "20v", "short_v2", f"{dist:.2f}cm_{now:%y%m%d-%H%M%S}.json")
outpath.parent.mkdir(exist_ok=True, parents=True)
data = {
    "fs": inFreq,
    "dist": dist,
    "data": [],
}

for i in range(100):
    ser = serial.Serial("/dev/ttyUSB0", 115200, timeout=1)
    try:
        anIn.configure(True, True)
        dev.triggerPC()
        ser.write(b'p')

        samples = []
        while True:
            status = anIn.status(True)
            if status in [pydwf.DwfState.Armed, pydwf.DwfState.Ready]:
                continue
            aSamples, lost, corrupt = anIn.statusRecord()

            if aSamples != 0:
                # cSamples = anIn.statusData(inCH, aSamples)
                cSamples = np.vstack(
                            [anIn.statusData(idx, aSamples)
                             for idx in inCH]).transpose()
                samples.append(cSamples)

            nSamples = sum(len(x) for x in samples)
            print(f"\r\033[K{i:>4}:  {100*nSamples/inNSamples: >4.2f}% {aSamples: >5} {nSamples: >9}", end="")

            if status == pydwf.DwfState.Done and nSamples >= inNSamples:
                break

        samples = np.concatenate(samples)[-inNSamples:]
        data["data"].append(samples.tolist())
        
    finally:
        ser.close()

# plt.close("all")
# fig, axd = plt.subplot_mosaic([
#     ["t"],
#     ["b"]
# ], sharex=True)

# x = np.arange(inNSamples) / inFreq
# for samples in data["data"]:
#     samples = np.array(samples)
#     axd["t"].plot(x, samples[:,1], c="C0", alpha=0.2)
#     axd["b"].scatter(x, samples[:,0], c="C1", alpha=0.2)

with open(outpath, "w+") as fd:
    json.dump(data, fd)
    
!espeak done

  99:  259.59% 16384     25959

# Close AD2 Device

In [ ]:
dev.close()